# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
#importing libraries
from sqlalchemy import create_engine
import re
import numpy as np
import pandas as pd
#downloading necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
#importing statements
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#importing from sklearn
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pickle
#the regular expression to detect a url
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ирина\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ирина\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ирина\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
#loading data from database
def load_data():
    engine = create_engine('sqlite:///etl_pipeline_db.db')
    df = pd.read_sql_table('messages', engine)
    X = df['message']
    Y = df.iloc[:,4:]
    return X, Y
X, Y = load_data()    

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    #getting list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    #replacing each url in text string with placeholder (the word "urlplaceholder")
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    #tokenizing text
    tokens = word_tokenize(text)
    #initiating lemmatizer
    lemmatizer = WordNetLemmatizer()
    #iterating through each token
    clean_tokens = []
    for tok in tokens:
        #lemmatizing, normalizing case, and removing leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
#building pipeline
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline

- Split data into train and test sets
- Train pipeline

In [5]:
#performing train/test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = .85,  test_size = .15)
#training classifier
train_data = pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
#predicting on test data
Y_pred = pipeline.predict(X_test)
#reporting the f1 score, precision and recall
for i in range(Y.shape[1]):
    print ('Category: "', Y.columns[i], '"')
    print(classification_report(Y_test.iloc[:,1].values, Y_pred[:,i]))    
    print ('====================================================')

Category: " related "
              precision    recall  f1-score   support

           0       0.98      0.09      0.16      3219
           1       0.19      0.99      0.32       686

    accuracy                           0.25      3905
   macro avg       0.58      0.54      0.24      3905
weighted avg       0.84      0.25      0.19      3905

Category: " request "
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      3219
           1       0.89      0.45      0.59       686

    accuracy                           0.89      3905
   macro avg       0.89      0.72      0.77      3905
weighted avg       0.89      0.89      0.88      3905

Category: " offer "
              precision    recall  f1-score   support

           0       0.82      1.00      0.90      3219
           1       0.00      0.00      0.00       686

    accuracy                           0.82      3905
   macro avg       0.41      0.50      0.45      3905
weighted a

D:\Programs\Anaconda3\envs\data-analyst\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.83      1.00      0.90      3219
           1       1.00      0.01      0.01       686

    accuracy                           0.83      3905
   macro avg       0.91      0.50      0.46      3905
weighted avg       0.86      0.83      0.75      3905

Category: " missing_people "
              precision    recall  f1-score   support

           0       0.82      1.00      0.90      3219
           1       0.00      0.00      0.00       686

    accuracy                           0.82      3905
   macro avg       0.41      0.50      0.45      3905
weighted avg       0.68      0.82      0.74      3905

Category: " refugees "
              precision    recall  f1-score   support

           0       0.82      1.00      0.90      3219
           1       0.00      0.00      0.00       686

    accuracy                           0.82      3905
   macro avg       0.41      0.50      0.45      3905
weighted avg       0.6

In [7]:
#calculating the accuracy of the model
accuracy = (Y_pred == Y_test).mean().mean()
print('Accuracy of the first model: ', accuracy)

Accuracy of the first model:  0.9455327927158914


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
#obtaining the parameters in the pipeline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x0000022F08D2D0D0>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

In [9]:
#specifying parameters for grid search
parameters = {'clf__estimator__n_estimators': [10, 20, 50]}
#creating grid search object
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, cv=3)

In [10]:
#training improved classifier
train_data_cv = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] clf__estimator__n_estimators=10 .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................. clf__estimator__n_estimators=10, total=  40.1s
[CV] clf__estimator__n_estimators=10 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   40.0s remaining:    0.0s


[CV] .................. clf__estimator__n_estimators=10, total=  39.0s
[CV] clf__estimator__n_estimators=10 .................................
[CV] .................. clf__estimator__n_estimators=10, total=  39.5s
[CV] clf__estimator__n_estimators=20 .................................
[CV] .................. clf__estimator__n_estimators=20, total= 1.1min
[CV] clf__estimator__n_estimators=20 .................................
[CV] .................. clf__estimator__n_estimators=20, total= 1.1min
[CV] clf__estimator__n_estimators=20 .................................
[CV] .................. clf__estimator__n_estimators=20, total= 1.1min
[CV] clf__estimator__n_estimators=50 .................................
[CV] .................. clf__estimator__n_estimators=50, total= 2.6min
[CV] clf__estimator__n_estimators=50 .................................
[CV] .................. clf__estimator__n_estimators=50, total= 2.5min
[CV] clf__estimator__n_estimators=50 .................................
[CV] .

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 13.0min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
#predicting on test data
Y_pred_cv = cv.predict(X_test)
#reporting the f1 score, precision and recall
for i in range(Y.shape[1]):
    print ('Category: "', Y.columns[i], '"')
    print(classification_report(Y_test.iloc[:,1].values, Y_pred_cv[:,i]))    
    print ('====================================================')

Category: " related "
              precision    recall  f1-score   support

           0       0.98      0.10      0.18      3219
           1       0.19      0.99      0.32       686

    accuracy                           0.26      3905
   macro avg       0.59      0.55      0.25      3905
weighted avg       0.84      0.26      0.20      3905

Category: " request "
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      3219
           1       0.89      0.43      0.58       686

    accuracy                           0.89      3905
   macro avg       0.89      0.71      0.76      3905
weighted avg       0.89      0.89      0.88      3905

Category: " offer "
              precision    recall  f1-score   support

           0       0.82      1.00      0.90      3219
           1       0.00      0.00      0.00       686

    accuracy                           0.82      3905
   macro avg       0.41      0.50      0.45      3905
weighted a

D:\Programs\Anaconda3\envs\data-analyst\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.83      0.99      0.90      3219
           1       0.56      0.08      0.14       686

    accuracy                           0.83      3905
   macro avg       0.70      0.53      0.52      3905
weighted avg       0.79      0.83      0.77      3905

Category: " clothing "
              precision    recall  f1-score   support

           0       0.82      1.00      0.90      3219
           1       0.00      0.00      0.00       686

    accuracy                           0.82      3905
   macro avg       0.41      0.50      0.45      3905
weighted avg       0.68      0.82      0.74      3905

Category: " money "
              precision    recall  f1-score   support

           0       0.83      1.00      0.90      3219
           1       1.00      0.01      0.01       686

    accuracy                           0.83      3905
   macro avg       0.91      0.50      0.46      3905
weighted avg       0.86      0.


Category: " other_weather "
              precision    recall  f1-score   support

           0       0.82      1.00      0.90      3219
           1       0.00      0.00      0.00       686

    accuracy                           0.82      3905
   macro avg       0.41      0.50      0.45      3905
weighted avg       0.68      0.82      0.74      3905

Category: " direct_report "
              precision    recall  f1-score   support

           0       0.89      0.99      0.93      3219
           1       0.86      0.41      0.56       686

    accuracy                           0.89      3905
   macro avg       0.87      0.70      0.75      3905
weighted avg       0.88      0.89      0.87      3905



In [12]:
#calculating the accuracy of the model
accuracy = (Y_pred_cv == Y_test).mean().mean()
print('Accuracy of the second - improved model: ', accuracy)

Accuracy of the second - improved model:  0.9456394935268175


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

We will use OneVsRestClassifier as the predictor in the model.

In [13]:
#building pipeline with OneVsRestClassifier
pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))])

In [14]:
#training classifier
train_data2 = pipeline2.fit(X_train, Y_train)

D:\Programs\Anaconda3\envs\data-analyst\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


In [15]:
#predicting on test data
Y_pred_p2 = pipeline2.predict(X_test)
#reporting the f1 score, precision and recall
for i in range(Y.shape[1]):
    print ('Category: "', Y.columns[i], '"')
    print(classification_report(Y_test.iloc[:,1].values, Y_pred_p2[:,i]))    
    print ('====================================================')

Category: " related "
              precision    recall  f1-score   support

           0       0.97      0.22      0.36      3219
           1       0.21      0.97      0.34       686

    accuracy                           0.35      3905
   macro avg       0.59      0.59      0.35      3905
weighted avg       0.84      0.35      0.35      3905

Category: " request "
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      3219
           1       0.78      0.61      0.69       686

    accuracy                           0.90      3905
   macro avg       0.85      0.79      0.81      3905
weighted avg       0.90      0.90      0.90      3905

Category: " offer "
              precision    recall  f1-score   support

           0       0.82      1.00      0.90      3219
           1       0.00      0.00      0.00       686

    accuracy                           0.82      3905
   macro avg       0.41      0.50      0.45      3905
weighted a

D:\Programs\Anaconda3\envs\data-analyst\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


 water "
              precision    recall  f1-score   support

           0       0.85      0.97      0.90      3219
           1       0.52      0.18      0.27       686

    accuracy                           0.83      3905
   macro avg       0.69      0.57      0.58      3905
weighted avg       0.79      0.83      0.79      3905

Category: " food "
              precision    recall  f1-score   support

           0       0.86      0.95      0.90      3219
           1       0.54      0.30      0.38       686

    accuracy                           0.83      3905
   macro avg       0.70      0.62      0.64      3905
weighted avg       0.81      0.83      0.81      3905

Category: " shelter "
              precision    recall  f1-score   support

           0       0.85      0.96      0.90      3219
           1       0.52      0.19      0.28       686

    accuracy                           0.83      3905
   macro avg       0.68      0.58      0.59      3905
weighted avg       0.79 


Category: " cold "
              precision    recall  f1-score   support

           0       0.82      0.99      0.90      3219
           1       0.11      0.01      0.01       686

    accuracy                           0.82      3905
   macro avg       0.47      0.50      0.46      3905
weighted avg       0.70      0.82      0.74      3905

Category: " other_weather "
              precision    recall  f1-score   support

           0       0.82      0.98      0.90      3219
           1       0.04      0.00      0.01       686

    accuracy                           0.81      3905
   macro avg       0.43      0.49      0.45      3905
weighted avg       0.68      0.81      0.74      3905

Category: " direct_report "
              precision    recall  f1-score   support

           0       0.91      0.94      0.92      3219
           1       0.66      0.56      0.61       686

    accuracy                           0.87      3905
   macro avg       0.79      0.75      0.77      390

In [16]:
#calculating the accuracy of the model
accuracy = (Y_pred_p2 == Y_test).mean().mean()
print('Accuracy of the third model: ', accuracy)

Accuracy of the third model:  0.9500711338739508


Accuracy of the last model has the highest value, so we will use this model in our project.

### 9. Export your model as a pickle file

In [18]:
pickle.dump(cv, open('model.pkl', "wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.